## Fundamentos de ETL com Python

- Extração
    - os dados são exraindos de diferentes fontes de dados
- Transformação
    - Onde são transformados e limpos
- Load
    - Carregar os dados no data warehouse

# Extração dos dados

### Importando os módulos

In [ ]:
import pandas as pd
import pandera as pa

#### Carregando arquivo csv

In [ ]:
# abrindo csv usando o separador ";"
df = pd.read_csv("./ocorrencia.csv", sep=";")
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,03/01/2010,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,03/01/2010,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,03/01/2010,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,04/01/2010,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,05/01/2010,19:25:00,0
...,...,...,...,...,...,...,...,...,...
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,15/08/2021,15:05:00,0
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,16/08/2021,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,17/08/2021,15:45:00,0
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,****,18/08/2021,16:00:00,0


#### Verificando os tipos de dados

In [ ]:
df.dtypes

codigo_ocorrencia            int64
codigo_ocorrencia2           int64
ocorrencia_classificacao    object
ocorrencia_cidade           object
ocorrencia_uf               object
ocorrencia_aerodromo        object
ocorrencia_dia              object
ocorrencia_hora             object
total_recomendacoes          int64
dtype: object

In [ ]:
# ocorrencia_dia é uma data mas foi carregada como object,então para trabalha com esse dado, tempo que converte
# para o tipo data
# vamos fazer isso usando o parse_dates no carregamento do csv
df = pd.read_csv("./ocorrencia.csv", sep=";",parse_dates=["ocorrencia_dia"])
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00,0
...,...,...,...,...,...,...,...,...,...
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,****,2021-08-18,16:00:00,0


In [ ]:
# verificando os tipos de dados de novo
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

#### Operações com data

- depois de converte o tipo data, podemos realizar as operações



In [ ]:
# acessando a coluna pelos meses 
df.ocorrencia_dia.dt.month

0       3
1       3
2       3
3       4
4       5
       ..
6109    8
6110    8
6111    8
6112    8
6113    8
Name: ocorrencia_dia, Length: 6114, dtype: int64

# Validação dos Dados
 - Checagem o conjunto de dados em relação aos tipos de dados estipulados para a leitura do conjunto de dados

In [ ]:
# imprimindo as primeiras 10 linhas
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-03-01,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-03-01,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-03-01,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-04-01,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-05-01,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,****,2010-06-01,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-07-01,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,****,2010-09-01,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-10-01,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,****,2010-10-01,20:00:00,0


In [ ]:
# acertando parametro dayfirstt=True, para que as datas fiquem no formato AAAA/MM/DD
df = pd.read_csv("./ocorrencia.csv", sep=";",parse_dates=["ocorrencia_dia"], dayfirst=True)
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,****,2021-08-18,16:00:00,0


In [ ]:
# imprimir as 10 ultimas linhas
df.tail(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6104,80238,80238,INCIDENTE GRAVE,VOTUPORANGA,SP,SDVG,2021-08-11,19:09:00,0
6105,80239,80239,INCIDENTE,BELO HORIZONTE,MG,SBBH,2021-08-11,19:58:00,0
6106,80249,80249,INCIDENTE,MANAUS,AM,SBEG,2021-08-14,15:40:00,0
6107,80243,80243,INCIDENTE,BELO HORIZONTE,MG,SBBH,2021-08-15,17:14:00,0
6108,80244,80244,INCIDENTE,BRAGANÇA PAULISTA,SP,SBBP,2021-08-15,15:20:00,0
6109,80245,80245,INCIDENTE,RECIFE,PE,SBRF,2021-08-15,15:05:00,0
6110,80247,80247,INCIDENTE,PORTO VELHO,RO,SBPV,2021-08-16,02:30:00,0
6111,80251,80251,INCIDENTE,BELÉM,PA,SBBE,2021-08-17,15:45:00,0
6112,80257,80257,ACIDENTE,DOM AQUINO,MT,****,2021-08-18,16:00:00,0
6113,80259,80259,INCIDENTE,CAMPINAS,SP,SBKP,2021-08-18,12:45:00,0


In [ ]:
# imprimindo os 10 primeiro elementos
# nas datas podem vir campos vazios e quando isso acontece, aparece o vlor NaT
# ja nso campos texto pode aparecer NaN
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,****,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,****,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,****,2010-01-10,20:00:00,0


Usando pandora

O schema levanta error parando a execução quando algum item nao passa na validação

In [ ]:
# montando schema
schema = pa.DataFrameSchema(
    columns = {
        "codigo":pa.Column(pa.Int, required=False),# o required diz que a coluna pode ou nao ta no dataframe
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String, pa.Check.str_length(2,2)), # definindo o tamanho do dado
        "ocorrencia_aerodromo":pa.Column(pa.String),
        "ocorrencia_dia":pa.Column(pa.DateTime), # Check.str_matches, usa regex para checa se o formato de hora esta correto
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),# permite que a coluna tenha valores Null
        "total_recomendacoes": pa.Column(pa.Int)
        
    }
)

Fazendo a validaçao do Dataframe

In [ ]:
schema.validate(df) # o error vai ser resolvido na etapa de limpeza de dados

SchemaError: ignored